# Reviewing Automated Machine Learning Explanations

As machine learning becomes more and more and more prevelant, the predictions made by models have greater influence over many aspects of our society. For example, machine learning models are an increasingly significant factor in how banks decide to grant loans or doctors prioritise treatments. The ability to interpret and explain models is increasingly important, so that the rationale for the predictions made by machine learning models can be explained and justified, and any inadvertant bias in the model can be identified.

When you use automated machine learning to train a model, you have the option to generate explanations of feature importance that quantify the extent to which each feature (including engineered features that were generated by preprocessing transformations) influences label prediction. In this lab, you'll explore the explanations generated by an automated machine learning experiment.


## Connect to Your Workspace

The first thing you need to do is to connect to your workspace using the Azure ML SDK.

> **Note**: If the authenticated session with your Azure subscription has expired since you completed the previous exercise, you'll be prompted to reauthenticate.

In [ ]:
import azureml.core
from azureml.core import Workspace

# Load the workspace from the saved config file
ws = Workspace.from_config()
print('Ready to use Azure ML {} to work with {}'.format(azureml.core.VERSION, ws.name))

## Run an Automated Machine Learning Experiment

To reduce time in this lab, you'll run an automated machine learning experiment on local compute with only three iterations.

Note that the **model_explainability** configuration option is set to **True**.

In [ ]:
import pandas as pd
from azureml.train.automl import AutoMLConfig
from azureml.core.experiment import Experiment
from azureml.widgets import RunDetails

# Load the data
train_data = pd.read_csv('data/diabetes.csv')

# Configure Auto ML
automl_config = AutoMLConfig(name='Automated ML Experiment',
                             task='classification',
                             compute_target='local',
                             training_data = train_data,
                             n_cross_validations = 2,
                             label_column_name='Diabetic',
                             iterations=3,
                             primary_metric = 'AUC_weighted',
                             max_concurrent_iterations=3,
                             model_explainability=True # Generate feature importance!
                             )

# Run the Auto ML experiment
automl_experiment = Experiment(ws, 'diabetes_automl')
automl_run = automl_experiment.submit(automl_config)
RunDetails(automl_run).show()

## View Feature Importance

When the experiment has completed in the widget above, click the run that produced the best result to see its details. Thens scroll to the bottom of the visualizations to see the relative feature importance.

You can also view feature importance for the best model produced by the experiment in [Azure machine learning studio](https://ml.azure.com) or by using the **ExplanationClient** class:

In [ ]:
from azureml.contrib.interpret.explanation.explanation_client import ExplanationClient

# Get the best model (2nd item in outputs)
best_run, fitted_model = automl_run.get_output()

# Get the feature explanations
client = ExplanationClient.from_run(best_run)
engineered_explanations = client.download_model_explanation()
feature_importances = engineered_explanations.get_feature_importance_dict()

# Overall feature importance
print('Feature\tImportance')
for key, value in feature_importances.items():
    print(key, '\t', value)

## View Importance for Auto-Engineered Features

Automated machine learning includes the option to try preprocessing the data, which often performs *feature engieering* to create new, derived features from which the model is trained. Let's enable that option and re-run the automated machine learning experiment.

In [ ]:
import pandas as pd
from azureml.train.automl import AutoMLConfig
from azureml.core.experiment import Experiment
from azureml.widgets import RunDetails

# Load the data
train_data = pd.read_csv('data/diabetes.csv')

# Configure Auto ML
automl_config = AutoMLConfig(name='Automated ML Experiment',
                             task='classification',
                             compute_target='local',
                             training_data = train_data,
                             n_cross_validations = 2,
                             label_column_name='Diabetic',
                             iterations=3,
                             primary_metric = 'AUC_weighted',
                             max_concurrent_iterations=3,
                             featurization='auto', # Enable feature engineering
                             model_explainability=True # Generate feature importance
                             )

# Run the Auto ML experiment
automl_experiment = Experiment(ws, 'diabetes_automl')
automl_run = automl_experiment.submit(automl_config)
RunDetails(automl_run).show()

Featurization is accomplished through the use of [Scikit-Learn transformation pipelines](https://scikit-learn.org/stable/modules/compose.html#combining-estimators) (not to be confused with Azure Machine Learning pipelines!). These produce models that include steps to transform the data before inferencing.

Run the following code to view the steps in the model pipeline, and the importance of the engineered features.

In [ ]:
from azureml.contrib.interpret.explanation.explanation_client import ExplanationClient

# Get the best model (2nd item in outputs)
best_run, fitted_model = automl_run.get_output()

print('Model pipeline steps:')
for step in fitted_model.named_steps:
    print('-',step)
    
# Get the feature explanations
client = ExplanationClient.from_run(best_run)
engineered_explanations = client.download_model_explanation(raw=False)
feature_importances = engineered_explanations.get_feature_importance_dict()

# Overall feature importance
print('\nFeature\tImportance')
for key, value in feature_importances.items():
    print(key, '\t', value)

> **More Information**: For more information Automated machine Learning, see the [Azure ML documentation](https://docs.microsoft.com/en-us/azure/machine-learning/service/how-to-configure-auto-train).